**Importing library**

In [ ]:

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import Flatten
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input, Conv2D, Dense, concatenate,MaxPooling2D,Dropout
from keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from collections import Counter
from keras.callbacks import EarlyStopping

**Load data**


In [ ]:
train_loc = 'independentimf/speakerindimf/train/'
validation_loc = 'independentimf/speakerindimf/val/'

In [ ]:
trdata = ImageDataGenerator(rescale=1 / 255.0,validation_split=0.20)
traindata = trdata.flow_from_directory(directory=train_loc, target_size=(224,224),batch_size=10,subset='training',shuffle=False)
validationdata = validationdata.flow_from_directory(directory=validation_loc, target_size=(224,224),subset='validation',batch_size=10,shuffle=False)


**Building the CNN**

In [ ]:
img_inputs = Input(shape=(224,224, 3))
# Step 1 - Convolution
classifier=Conv2D(64, (5, 5),  activation = 'relu')(img_inputs)

# Step 2 - Pooling
classifier=MaxPooling2D(pool_size = (2, 2))(classifier)
# Step 1 - Convolution
classifier=Conv2D(64, (3, 3), activation = 'relu')(classifier)

# Step 2 - Pooling
classifier=MaxPooling2D(pool_size = (2, 2))(classifier)
# Step 1 - Convolution
classifier=Conv2D(96, (3, 3),  activation = 'relu')(classifier)

# Step 2 - Pooling
classifier=MaxPooling2D(pool_size = (2, 2))(classifier)
# Step 1 - Convolution
classifier=Conv2D(96, (3, 3),  activation = 'relu')(classifier)

# Step 2 - Pooling
classifier=MaxPooling2D(pool_size = (2, 2))(classifier)


#classifier=MaxPooling2D(pool_size = (2, 2))(classifier)
# Step 3 - Flattening
classifier=Flatten()(classifier)

# Step 4 - Full connection
classifier=Dense(units = 256, activation = 'relu')(classifier)
classifier=Dropout(0.6)(classifier)
classifier=Dense(units = 128, activation = 'relu')(classifier)
classifier=Dropout(0.3)(classifier)
classifier=Dense(units = 64, activation = 'relu')(classifier)
classifier=Dropout(0.15)(classifier)
classifier=Dense(units = 32, activation = 'relu')(classifier)
classifier=Dropout(0.075)(classifier)
classifier=Dense(units = 16, activation = 'relu')(classifier)
classifier=Dropout(0.0325)(classifier)
classifier=Dense(units = 8, activation = 'relu')(classifier)
outputs=Dense(units = 6, activation = 'softmax')(classifier)

model = Model(inputs=img_inputs, outputs=outputs, name="LS_model")





**Compiling the CNN**

In [ ]:
opt=opt = keras.optimizers.SGD(learning_rate=0.00001)

model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

STEP_SIZE_TRAIN=traindata.n//traindata.batch_size

STEP_SIZE_VALID=testdata.n//testdata.batch_size

checkpoint = ModelCheckpoint("working/best_model.h5", monitor='val_accuracy', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='auto')

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
model.summary()

**Fitting the CNN to the images**

In [ ]:
history = model.fit_generator(traindata,
                    epochs =600,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    #class_weight=class_weights,
                    validation_data = testdata,
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=[checkpoint,early]
                    )

**Visualize history**

import matplotlib.pyplot as plt

plt.plot(hist.history['accuracy'], label='train')
plt.plot(hist.history['val_accuracy'], label='val')
plt.title('Accuracy and Validation Accuracy')
plt.legend();
plt.show()